<a href="https://colab.research.google.com/github/Alen0101/PreddiplomnayaPraktika/blob/main/PreddiplomnayaPraktika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [3]:
!pip install numpy

In [4]:
!pip install pandas

In [5]:
!pip install matplotlib

In [6]:
import tensorflow
import keras
import numpy
import pandas
import matplotlib

In [10]:
pip install kaggle

In [12]:
import os

os.environ["KAGGLE_USERNAME"] = "alenphil"
os.environ["KAGGLE_KEY"]      = "KGAT_7c67e205d53e94a2be0d353053710006"

In [13]:
dataset_slug = "kmader/food41"
!kaggle datasets download -d {dataset_slug} -p ./data --unzip

Dataset URL: https://www.kaggle.com/datasets/kmader/food41
License(s): copyright-authors
100% 5.30G/5.30G [01:27<00:00, 244MB/s]
100% 5.30G/5.30G [01:27<00:00, 64.9MB/s]
